## Solución del problema

### 1.Instalación e importación de librerias

In [30]:
# Manejo de gráficas
!pip install scikit-plot

In [31]:
# Manejo de gráficas
!pip install scikit-plot
#Manejo de lemmatization spanish
!pip install stanza

In [32]:
# librería Natural Language Toolkit, usada para trabajar con textos
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ascas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ascas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
#El lemmatizer de NLTK NO funciona en español, por lo que se usará el de Stanza
# Descarga de paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabra
# ¿Qué es el lema de una palabra? ¿Qué tan dificil puede ser obtenerlo, piensa en el caso en que tuvieras que escribir la función que realiza esta tarea?
#nltk.download('wordnet')
import stanza
stanza.download('es') # descarga el modelo de lematización en español
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')

2024-04-02 01:55:08 INFO: Downloaded file to C:\Users\ascas\stanza_resources\resources.json
2024-04-02 01:55:08 INFO: Downloading default packages for language: es (Spanish) ...
2024-04-02 01:55:10 INFO: File exists: C:\Users\ascas\stanza_resources\es\default.zip
2024-04-02 01:55:15 INFO: Finished downloading models and saved to C:\Users\ascas\stanza_resources
2024-04-02 01:55:15 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-02 01:55:15 INFO: Downloaded file to C:\Users\ascas\stanza_resources\resources.json
2024-04-02 01:55:16 INFO: Loading these models for language: es (Spanish):
| Processor | Package         |
-------------------------------
| tokenize  | ancora          |
| mwt       | ancora          |
| pos       | ancora_charlm   |
| lemma     | ancora_nocharlm |

2024-04-02 01:55:16 INFO: Using device: cpu
2024-04-02 01:55:16 INFO: Loading: tokenize
2024-04-02 01:55:16 INFO: Loading: mwt
2024-04-02 01:55:16 INFO: Loading: pos
2024-04-02 01:55:16 INFO: Loading: lemma
2024-04-02 01:55:16 INFO: Done loading processors!


In [35]:
# Instalación de librerias
import pandas as pd
import numpy as np
import sys

#Usados en el proyecto 1
from nltk.tokenize import WordPunctTokenizer 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer



######################################



#!{sys.executable} -m pip install pandas-profiling

import re, string, unicodedata

#from nltk import word_tokenize, sent_tokenize


from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

### 2. Perfilamiento y entendimiento de los datos

#### 2.1 Lectura de los Datos

In [36]:
# Uso de la libreria pandas para la lectura de archivos
data=pd.read_csv('train_reviews.csv', sep=',', encoding = 'utf-8')
# Asignación a una nueva variable de los datos leidos
data_train=data

In [37]:
data_train

,Review,Class
0,Muy buena atención y aclaración de dudas por p...,5
1,Buen hotel si están obligados a estar cerca de...,3
2,"Es un lugar muy lindo para fotografías, visite...",5
3,Abusados con la factura de alimentos siempre s...,3
4,Tuvimos un par de personas en el grupo que rea...,3
...,...,...
7870,"Me parece buen sistema, agiliza el transporte,...",4
7871,"Fue una escapada de un día desde el complejo, ...",4
7872,La Plaza de la Revolución es un lugar emblemát...,3
7873,Es la segunda ocasión que me quedo en los cuar...,1


In [38]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7875 entries, 0 to 7874
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  7875 non-null   object
 1   Class   7875 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 123.2+ KB


## 2.2 Entendimiento de los datos

In [39]:
from scipy import stats as st

textos = data_train.copy()
textos['Conteo'] = [len(x) for x in textos['Review']]

#Por ahora: La moda no da información relevante
"""def moda(textos):
    for i in textos['Review']: 
        dict = {}
        for x in i.split(' '): 
            print(i)
        
            if x in dict.keys():
                dict[x] += 1
            else:
                dict[x] = 1

        max_key = max(dict, key=dict.get)
        print(max_key)


moda(textos)
"""
#textos['Moda'] =
#Max tiene el máximo tamaño de la palabra
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['Review']]
#Max tiene el minimo tamaño de la palabra
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['Review']]


In [40]:
textos

,Review,Class,Conteo,Max,Min
0,Muy buena atención y aclaración de dudas por p...,5,252,20,0
1,Buen hotel si están obligados a estar cerca de...,3,297,13,1
2,"Es un lugar muy lindo para fotografías, visite...",5,104,12,1
3,Abusados con la factura de alimentos siempre s...,3,422,14,1
4,Tuvimos un par de personas en el grupo que rea...,3,419,14,1
...,...,...,...,...,...
7870,"Me parece buen sistema, agiliza el transporte,...",4,113,11,2
7871,"Fue una escapada de un día desde el complejo, ...",4,261,12,2
7872,La Plaza de la Revolución es un lugar emblemát...,3,777,14,1
7873,Es la segunda ocasión que me quedo en los cuar...,1,651,12,0


In [41]:
"""import ydata_profiling
from ydata_profiling import ProfileReport
ProfileReport(textos)"""

'import ydata_profiling\nfrom ydata_profiling import ProfileReport\nProfileReport(textos)'

### 2.3 Preparación de los datos

#### 2.3.1 Limpieza de los datos

##### 2.3.1.1 Duplicados

In [42]:
textos.duplicated(keep = False).sum()

102

In [43]:
textos.drop_duplicates(keep='first', inplace=True)
textos.duplicated(keep = False).sum()

0

In [44]:
spanish_stopwords = stopwords.words('spanish')
print(spanish_stopwords)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
          new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
          new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_words.append(word.lower())
    return new_words
    

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

#def replace_numbers(words):
#    """Replace all interger occurrences in list of tokenized words with textual representation"""
#    p = inflect.engine()
#    print(words)
#    new_words = []
#    for word in words:
#        if word.isdigit():
#            new_word = p.number_to_words(word)
#            new_words.append(new_word)
#            print("if " + new_word)
#        else:
#            new_words.append(word)
#    return new_words

def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in spanish_stopwords:
            new_words.append(word)
    return new_words

def preprocessing(words):
    words = to_lowercase(words)
 #   words = replace_numbers(words)
    words = remove_punctuation(words)
#    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

#### 2.3.2 Tokenización

Se utiliza WordPunctTokenizer dado que el idioma español no presenta contracciones y este Tokenizer es más robusto frente a los signos de puntuación. 

In [45]:

textos['tokens'] = [ WordPunctTokenizer().tokenize(i) for i in textos['Review']]
    
textos.iloc[1199]['tokens']

['El',
 'día',
 'lunes',
 '17',
 'de',
 'setiembre',
 '2012',
 'visite',
 'con',
 'mi',
 'familia',
 'Monserratte',
 'y',
 'debía',
 'pagar',
 'un',
 'taxi',
 ',',
 'por',
 'lo',
 'que',
 'a',
 'las',
 '12',
 'M',
 '.',
 'D',
 ',',
 'fui',
 'a',
 'la',
 'taquilla',
 'para',
 'cambiar',
 '50',
 ',',
 '000',
 'pesos',
 'y',
 'poder',
 'pagar',
 'mi',
 'taxi',
 '.',
 'Mi',
 'sorpresa',
 'al',
 'pagar',
 'el',
 'taxi',
 'fue',
 'que',
 'le',
 'entregue',
 'los',
 '50',
 ',',
 '000',
 'al',
 'empleado',
 'en',
 'la',
 'taquilla',
 'y',
 'me',
 'estafó',
 ',',
 'me',
 'dió',
 'un',
 'billete',
 'de',
 '20',
 ',',
 '000',
 'encima',
 'y',
 'debajo',
 'venia',
 'uno',
 'de',
 '2',
 ',',
 '000',
 'y',
 'luego',
 'el',
 'otro',
 'de',
 '1',
 ',',
 '000',
 '.',
 'El',
 'funcionario',
 'de',
 'la',
 'taquilla',
 'me',
 'estafó',
 'con',
 'el',
 'cambio',
 'y',
 'se',
 'dejó',
 '27',
 ',',
 '000',
 'pesos',
 '.',
 'Es',
 'bueno',
 'que',
 'sepan',
 'que',
 'tienen',
 'un',
 'ladrón',
 'trabajando',

In [46]:
textos['tokens'].dropna()

0       [Muy, buena, atención, y, aclaración, de, duda...
1       [Buen, hotel, si, están, obligados, a, estar, ...
2       [Es, un, lugar, muy, lindo, para, fotografías,...
3       [Abusados, con, la, factura, de, alimentos, si...
4       [Tuvimos, un, par, de, personas, en, el, grupo...
                              ...                        
7870    [Me, parece, buen, sistema, ,, agiliza, el, tr...
7871    [Fue, una, escapada, de, un, día, desde, el, c...
7872    [La, Plaza, de, la, Revolución, es, un, lugar,...
7873    [Es, la, segunda, ocasión, que, me, quedo, en,...
7874    [Llegamos, por, casualidad, a, Los, Mercaderes...
Name: tokens, Length: 7802, dtype: object

#### 2.3.2 Eliminación de ruido
Esta eliminación del ruido implica eliminar aquello asociado a mayusculas, eliminación de puntuación y de elementos NO ascci 

In [47]:
textos['tokens1']=textos['tokens'].apply(preprocessing) #Aplica la eliminación del ruido

textos.head()

,Review,Class,Conteo,Max,Min,tokens,tokens1
0,Muy buena atención y aclaración de dudas por p...,5,252,20,0,"[Muy, buena, atención, y, aclaración, de, duda...","[buena, atención, aclaración, dudas, parte, se..."
1,Buen hotel si están obligados a estar cerca de...,3,297,13,1,"[Buen, hotel, si, están, obligados, a, estar, ...","[buen, hotel, si, obligados, cerca, centro, co..."
2,"Es un lugar muy lindo para fotografías, visite...",5,104,12,1,"[Es, un, lugar, muy, lindo, para, fotografías,...","[lugar, lindo, fotografías, visiten, selina, m..."
3,Abusados con la factura de alimentos siempre s...,3,422,14,1,"[Abusados, con, la, factura, de, alimentos, si...","[abusados, factura, alimentos, siempre, echan,..."
4,Tuvimos un par de personas en el grupo que rea...,3,419,14,1,"[Tuvimos, un, par, de, personas, en, el, grupo...","[par, personas, grupo, realmente, queríamos, c..."


#### 2.3.3 Normalización
El stemming se opata por hacer a traves del método SnowballStemmer el cual esta enfocado hacia el idioma español

In [48]:
stemmer = SnowballStemmer('spanish')

textos['tokens'] =  [ [stemmer.stem(word) for word in tokens] for tokens in textos['tokens']]
i = 1201
print(textos['tokens'][i])
print(textos['Review'][i])

['el', 'dia', 'lun', '17', 'de', 'setiembr', '2012', 'visit', 'con', 'mi', 'famili', 'monserratt', 'y', 'deb', 'pag', 'un', 'taxi', ',', 'por', 'lo', 'que', 'a', 'las', '12', 'm', '.', 'd', ',', 'fui', 'a', 'la', 'taquill', 'par', 'cambi', '50', ',', '000', 'pes', 'y', 'pod', 'pag', 'mi', 'taxi', '.', 'mi', 'sorpres', 'al', 'pag', 'el', 'taxi', 'fue', 'que', 'le', 'entreg', 'los', '50', ',', '000', 'al', 'emple', 'en', 'la', 'taquill', 'y', 'me', 'estaf', ',', 'me', 'dio', 'un', 'billet', 'de', '20', ',', '000', 'encim', 'y', 'debaj', 'veni', 'uno', 'de', '2', ',', '000', 'y', 'lueg', 'el', 'otro', 'de', '1', ',', '000', '.', 'el', 'funcionari', 'de', 'la', 'taquill', 'me', 'estaf', 'con', 'el', 'cambi', 'y', 'se', 'dej', '27', ',', '000', 'pes', '.', 'es', 'buen', 'que', 'sep', 'que', 'tien', 'un', 'ladron', 'trabaj', 'en', 'sus', 'instal', 'y', 'asi', 'com', 'fui', 'estaf', 'yo', ',', 'me', 'imagin', 'que', 'tambien', 'otros', 'turist', 'pas', 'por', 'lo', 'mism', '.', 'revis', 'muy'

In [49]:
"""docs = [ nlp(review) for review in textos['Review']]
textos['lemas']= [[[word.lemma  for word in sent.words] for sent in doc.sentences] for doc in docs]


def oneSentence(list):
    complete = []
    for sent in list:
        for word in sent:
            complete.append(word)

    return complete

textos['lemas'] = [oneSentence(i) for i in textos['lemas']]

"""

"docs = [ nlp(review) for review in textos['Review']]\ntextos['lemas']= [[[word.lemma  for word in sent.words] for sent in doc.sentences] for doc in docs]\n\n\ndef oneSentence(list):\n    complete = []\n    for sent in list:\n        for word in sent:\n            complete.append(word)\n\n    return complete\n\ntextos['lemas'] = [oneSentence(i) for i in textos['lemas']]\n\n"

Este análisis se demora bastante en ejecutar, por lo que se decide guardar estos tokens en un archivo csv intermedio 

In [50]:
#Hacer código para guardar los lemas en csv

#### 2.3.4 Selección de campos 
-> Separación palabras ( tokens) y categoria 

In [51]:
textos.head()

,Review,Class,Conteo,Max,Min,tokens,tokens1
0,Muy buena atención y aclaración de dudas por p...,5,252,20,0,"[muy, buen, atencion, y, aclar, de, dud, por, ...","[buena, atención, aclaración, dudas, parte, se..."
1,Buen hotel si están obligados a estar cerca de...,3,297,13,1,"[buen, hotel, si, estan, oblig, a, estar, cerc...","[buen, hotel, si, obligados, cerca, centro, co..."
2,"Es un lugar muy lindo para fotografías, visite...",5,104,12,1,"[es, un, lug, muy, lind, par, fotograf, ,, vis...","[lugar, lindo, fotografías, visiten, selina, m..."
3,Abusados con la factura de alimentos siempre s...,3,422,14,1,"[abus, con, la, factur, de, aliment, siempr, s...","[abusados, factura, alimentos, siempre, echan,..."
4,Tuvimos un par de personas en el grupo que rea...,3,419,14,1,"[tuv, un, par, de, person, en, el, grup, que, ...","[par, personas, grupo, realmente, queríamos, c..."


In [52]:
textos['tokens'] = textos['tokens'].apply(lambda x: ' '.join(map(str, x)))
textos.head()

,Review,Class,Conteo,Max,Min,tokens,tokens1
0,Muy buena atención y aclaración de dudas por p...,5,252,20,0,muy buen atencion y aclar de dud por part de l...,"[buena, atención, aclaración, dudas, parte, se..."
1,Buen hotel si están obligados a estar cerca de...,3,297,13,1,buen hotel si estan oblig a estar cerc del cen...,"[buen, hotel, si, obligados, cerca, centro, co..."
2,"Es un lugar muy lindo para fotografías, visite...",5,104,12,1,"es un lug muy lind par fotograf , visit selin ...","[lugar, lindo, fotografías, visiten, selina, m..."
3,Abusados con la factura de alimentos siempre s...,3,422,14,1,abus con la factur de aliment siempr se echan ...,"[abusados, factura, alimentos, siempre, echan,..."
4,Tuvimos un par de personas en el grupo que rea...,3,419,14,1,tuv un par de person en el grup que realment q...,"[par, personas, grupo, realmente, queríamos, c..."


In [53]:
textos['tokens1'] = textos['tokens1'].apply(lambda x: ' '.join(map(str, x)))
textos.head()

,Review,Class,Conteo,Max,Min,tokens,tokens1
0,Muy buena atención y aclaración de dudas por p...,5,252,20,0,muy buen atencion y aclar de dud por part de l...,buena atención aclaración dudas parte señorita...
1,Buen hotel si están obligados a estar cerca de...,3,297,13,1,buen hotel si estan oblig a estar cerc del cen...,buen hotel si obligados cerca centro convencio...
2,"Es un lugar muy lindo para fotografías, visite...",5,104,12,1,"es un lug muy lind par fotograf , visit selin ...",lugar lindo fotografías visiten selina música ...
3,Abusados con la factura de alimentos siempre s...,3,422,14,1,abus con la factur de aliment siempr se echan ...,abusados factura alimentos siempre echan culpa...
4,Tuvimos un par de personas en el grupo que rea...,3,419,14,1,tuv un par de person en el grup que realment q...,par personas grupo realmente queríamos comer c...


# Segundo profile tras aplicación de preprocessing

In [54]:
import ydata_profiling
from ydata_profiling import ProfileReport
ProfileReport(textos)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Analisis corto de lo que sucede tras el preprocessing, palabras más relevantes etc...

In [55]:
X_data, y_data = textos['tokens1'],textos['Class']
y_data

0       5
1       3
2       5
3       3
4       3
       ..
7870    4
7871    4
7872    3
7873    1
7874    5
Name: Class, Length: 7802, dtype: int64

## 3. Embedding del texto: 
Transformación del texto en formato para la máquina

#### 3.1 Count Vectorizer
Utilización del conteo de palabras presentes como Vectorizer -> No creo que sea tan bueno

In [56]:
count = CountVectorizer()
X_countVectorizer = count.fit_transform(X_data)
print(X_countVectorizer.shape)
X_countVectorizer.toarray()[3]

(7802, 25397)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### 3.1 TfiDf

In [57]:
tfidf = TfidfVectorizer()
X_tfidfVectorizer = tfidf.fit_transform(X_data)
print(X_tfidfVectorizer.shape)
X_tfidfVectorizer.toarray()[3]

(7802, 25397)


array([0., 0., 0., ..., 0., 0., 0.])

In [58]:
df = pd.DataFrame(X_tfidfVectorizer[0].T.todense(),
    	index=tfidf.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
df.head(20)

,TF-IDF
verónica,0.313710
laborable,0.313710
aclaración,0.299987
habitaciónes,0.290251
ordenadas,0.276528
parte,0.254494
atenta,0.231050
señorita,0.224392
cuestión,0.222057
dudas,0.213178


### 4. Construcción de los modelos

Recursos utilizados: 
https://medium.com/escueladeinteligenciaartificial/procesamiento-de-lenguaje-natural-stemming-y-lemmas-f5efd90dca8
https://neptune.ai/blog/vectorization-techniques-in-nlp-guide
https://gitlab.virtual.uniandes.edu.co/ISIS3301/practicas/blob/master/ProcesamientoTextos/Preparaci%C3%B3n_de_textos_estudiante.ipynb
